<a href="https://colab.research.google.com/github/jdanifalcon/scrapeo_SMN/blob/main/parte1smn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Descargar la base de datos del Sistema Metereologico Nacional



In [14]:
# Cargar las librerías necesarias
library(rvest)    # Nos ayuda con el scrapping del HTML
library(stringr)   # Nos ayuda a manipular cadenas de texto

In [2]:
# Definir acrónimos de estados y la URL base
estados <- c("ags", "bc", "bcs", "camp", "coah", "col", "chis", "chih",
              "cdmx", "dgo", "gto","gro", "hgo", "jal", "mex", "mich", "mor",
              "nay", "nl", "oax", "pue", "qro","qroo", "slp", "sin", "son", "tab",
              "tamps", "tlax", "ver", "yuc", "zac")

url_base <- "https://smn.conagua.gob.mx/tools/RESOURCES/Normales_Climatologicas/catalogo/cat_"

In [3]:
# Crear una función para obtener la tabla de un estado
obtener_datos_estado <- function(estado) {
  url <- paste0(url_base, estado, ".html")
  html <- read_html(url)
  tabla <- html_nodes(html, "table") %>% html_table(fill = TRUE)
  df <- as.data.frame(tabla[[1]][3:nrow(tabla[[1]]), ])
  return(df)
}

In [4]:
# Ejemplo: Obtener datos del estado de Tabasco (posición 27 en la lista)
estado_tabasco <- obtener_datos_estado(estados[27])

# Imprimir los primeros registros del estado de Tabasco
head(estado_tabasco)

,X1,X2,X3,X4,X5,X6,X7,X8,X9,X10,X11
,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
1,27001,Balancan De Dominguez (SMN),Balancán,Suspendida,Diario,Mensual,Ext-hist,,,,
2,27002,Benito Juarez,Centla,Operando,Diario,Mensual,Ext-hist,,,,
3,27003,Blasillo,Huimanguillo,Operando,Diario,Mensual,Ext-hist,,1971-2000,1981-2010,1991-2020
4,27004,Boca Del Cerro,Tenosique,Operando,Diario,Mensual,Ext-hist,1961-1990,1971-2000,1981-2010,1991-2020
5,27006,Buenavista,Balancán,Operando,Diario,Mensual,Ext-hist,,1971-2000,1981-2010,1991-2020
6,27007,Centro Experimental W-75,Cárdenas,Operando,Diario,Mensual,Ext-hist,1961-1990,,1981-2010,1991-2020


In [5]:
# Ejemplo: Obtener datos del estado de Chiapas (posición 07 en la lista)
estado_chiapas <- obtener_datos_estado(estados[07])

# Imprimir los primeros registros del estado de Chiapas
head(estado_chiapas)

,X1,X2,X3,X4,X5,X6,X7,X8,X9,X10,X11
,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
1,7001,Abasolo Chiapas (CFE),Ocosingo,Suspendida,Diario,Mensual,Ext-hist,,1971-2000,1981-2010,
2,7002,Abelardo L. Rodríguez,Comitán De Domínguez,Suspendida,Diario,Mensual,Ext-hist,1961-1990,1971-2000,1981-2010,1991-2020
3,7003,Acala,Acala,Operando,Diario,Mensual,Ext-hist,1961-1990,1971-2000,1981-2010,1991-2020
4,7004,Agua Azul,Ocosingo,Suspendida,Diario,Mensual,Ext-hist,,,,
5,7005,Almandro (CFE),Huitiupán,Suspendida,Diario,Mensual,Ext-hist,1961-1990,,,
6,7006,Altamirano (SMN),Altamirano,Operando,Diario,Mensual,Ext-hist,1961-1990,1971-2000,1981-2010,1991-2020


In [6]:
procesar_estados <- function(estado_indices) {
  # Inicializar DataFrame vacío
  df <- NULL

  # Procesar cada estado según su índice en la lista de `estados`
  for (estado_idx in estado_indices) {
    # Obtener el acrónimo del estado
    estado <- estados[estado_idx]

    # Crear una carpeta para cada estado dentro de la carpeta destino
    path_estado <- paste0("destino/", estado)
    if (!file.exists(path_estado)) {
      dir.create(path_estado, recursive = TRUE)
    }

    # Construir la URL completa
    url_completa <- paste0(url_base, estado, ".html")

    # Obtener los datos de la tabla HTML
    df_estado <- obtener_datos_estado(estado)

    # Imprimir los primeros registros del estado
    head(df_estado)

    # Recorrer cada estación del estado para descargar el archivo
    for (i in 1:nrow(df_estado)) {
      # Asegurar que el número de estación tenga 5 dígitos, agregando ceros al inicio si es necesario
      num_estacion <- sprintf("%05d", as.numeric(df_estado[i, "X1"]))

      # Construir la URL para la estación específica con el formato correcto
      urldata <- paste0("https://smn.conagua.gob.mx/tools/RESOURCES/Normales_Climatologicas/Mensuales/",
                        estado, "/mes", num_estacion, ".txt")

      # Definir el archivo de destino en la carpeta del estado
      destfile <- paste0(path_estado, "/mes", num_estacion, ".txt")

      # Descargar el archivo
      tryCatch({
        download.file(urldata, destfile)
        print(paste("Descargado:", urldata))
      }, error = function(e) {
        # Si hay un error, imprimir un mensaje y continuar
        print(paste("Error en la descarga de:", urldata))
      })

      # Esperar un tiempo aleatorio entre descargas para no ser bloqueado
      Sys.sleep(5 * runif(1))
    }
  }

  # Devolver el DataFrame procesado
  return(df)
}

In [7]:
# Llamar a la función para los estados de Chiapas (7) y Tabasco (27)
df_combinado <- procesar_estados(c(27, 7))

[1] "Descargado: https://smn.conagua.gob.mx/tools/RESOURCES/Normales_Climatologicas/Mensuales/tab/mes27001.txt"
[1] "Descargado: https://smn.conagua.gob.mx/tools/RESOURCES/Normales_Climatologicas/Mensuales/tab/mes27002.txt"
[1] "Descargado: https://smn.conagua.gob.mx/tools/RESOURCES/Normales_Climatologicas/Mensuales/tab/mes27003.txt"
[1] "Descargado: https://smn.conagua.gob.mx/tools/RESOURCES/Normales_Climatologicas/Mensuales/tab/mes27004.txt"
[1] "Descargado: https://smn.conagua.gob.mx/tools/RESOURCES/Normales_Climatologicas/Mensuales/tab/mes27006.txt"
[1] "Descargado: https://smn.conagua.gob.mx/tools/RESOURCES/Normales_Climatologicas/Mensuales/tab/mes27007.txt"
[1] "Descargado: https://smn.conagua.gob.mx/tools/RESOURCES/Normales_Climatologicas/Mensuales/tab/mes27008.txt"
[1] "Descargado: https://smn.conagua.gob.mx/tools/RESOURCES/Normales_Climatologicas/Mensuales/tab/mes27009.txt"
[1] "Descargado: https://smn.conagua.gob.mx/tools/RESOURCES/Normales_Climatologicas/Mensuales/tab/mes270